In [21]:
import pandas as pd
import numpy as np
import re
import time
import bs4 as bs4
import json
import glob
import tqdm

pd.set_option('max.columns',131)
%matplotlib inline
%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [8]:
df = pd.read_csv('./data_labels_old.csv',index_col=0).dropna(subset=['y'])

In [12]:
df.shape

(1164, 16)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# 1.0 - Data Cleaning

In [15]:
df_clean = pd.DataFrame(index=df.index)
df_clean['title'] = df['watch-title']

In [22]:
clean_date = df['watch-time-text'].str.extract(r'(\d+) de ([a-z]+)\. de (\d+)')
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)

month_map = {
    'jan':'Jan',
    'fev':'Feb',
    'mar':'Mar',
    'abr':'Apr',
    'mai':'May',
    'jun':'Jun',
    'jul':'Jul',
    'ago':'Aug',
    'set':'Sep',
    'out':'Oct',
    'nov':'Nov',
    'dez':'Dec'
}

clean_date[1] = clean_date[1].map(month_map)
clean_date = clean_date.apply(lambda x: ' '.join(x), axis=1)
df_clean['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

In [24]:
## Clean View Count
views = df['watch-view-count'].str.extract(r'(\d+\.?\d*)', expand=False).str.replace('.','').fillna(0).astype(int)
df_clean['views'] = views

# 2.0 - Features

In [25]:
features = pd.DataFrame(index=df_clean.index)
y = df['y'].copy()

In [26]:
features['time_since_pub'] = (pd.to_datetime('2019-12-03') - df_clean['date'])/np.timedelta64(1,'D')
features['views'] = df_clean.views
features['views_per_day'] = features.views/features.time_since_pub
features.drop(['time_since_pub'],axis=1,inplace=True)

In [28]:
features.shape

(1164, 2)

In [29]:
features.head()

,views,views_per_day
0,28028,61.464912
394,1161,21.109091
393,141646,809.405714
392,325,21.666667
391,61,7.625000


In [30]:
mask_train = df_clean.date <'2019-04-01'
mask_val = df_clean.date>='2019-04-01'

In [31]:
Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((555, 2), (609, 2), (555,), (609,))

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [63]:
title_train = df_clean[mask_train].title
title_val = df_clean[mask_val].title

title_vec = TfidfVectorizer(min_df=2, ngram_range=(1,3))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [64]:
from scipy.sparse import hstack, vstack

In [65]:
title_bow_train.shape

(555, 1144)

In [66]:
Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [67]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((555, 1146), (609, 1146))

In [68]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf=1, class_weight='balanced')
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=1000, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [69]:
p = mdl.predict_proba(Xval_wtitle)[:,1]

In [70]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [72]:
print('Results w/ min_samples_leaf=1, 1k trees and ngram_range=(1,3)\n')
print('ap:',average_precision_score(yval,p))
print('auc:',roc_auc_score(yval,p))

Results w/ min_samples_leaf=1, 1k trees and ngram_range=(1,3)

ap: 0.22228951304206077
auc: 0.6914990859232175


# LightGBM

In [51]:
from lightgbm import LGBMClassifier

In [73]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])